In [1]:
!pip install spacy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grab-screen 0.0.13 requires click==6.7, but you have click 8.0.3 which is incompatible.
grab-screen 0.0.13 requires requests==2.18.4, but you have requests 2.26.0 which is incompatible.


  Attempting uninstall: click
    Found existing installation: click 6.7
    Uninstalling click-6.7:
      Successfully uninstalled click-6.7


In [2]:
!python -m spacy download en_core_web_sm

2021-11-15 13:16:56.410596: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [24]:
import spacy
import pandas as pd
from textblob import TextBlob
import numpy as np

In [4]:
dota_2_reviews = pd.read_csv('dota-metacritics.csv')

In [31]:
genshin = pd.read_csv('genshin-googleplay-reviews.csv')

In [32]:
genshin.head()

,Unnamed: 0,reviews,score,label
0,0,Amazing grapics. Amazing gameplay. Amazing cha...,5,1
1,1,"This game is absolutely amazing to play, but n...",3,0
2,2,"Great game, beautiful graphics, and very enjoy...",3,0
3,3,The open-world experience is notable for a mob...,3,0
4,4,"It's a awesome game, and seeing as it's gettin...",3,0


In [19]:
dota_2_reviews.head(5)

,review,score
0,Great game ruined by terrible company that is ...,10
1,After almost 1 year or more since it got out o...,5
2,Re-review. I tried to get into Dota 2. I can g...,0
3,I hoped this game would get me really hooked u...,0
4,This game improved so much since it was releas...,10


In [10]:
dota_2_reviews.drop('Unnamed: 0', axis=1, inplace=True)

In [12]:
sentences = dota_2_reviews['review']

In [25]:
sample_sentences = np.array(dota_2_reviews['review'].head(5))

In [13]:
nlp = spacy.load("en_core_web_sm")

In [29]:
for sentence in sample_sentences:
    doc = nlp(sentence)
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_,
      token.pos_,[child for child in token.children])

Great amod game NOUN ADJ []
game nsubjpass involved VERB NOUN [Great, ruined]
ruined acl game NOUN VERB [by]
by agent ruined VERB ADP [company]
terrible amod company NOUN ADJ []
company pobj by ADP NOUN [terrible]
that nsubj is AUX PRON []
is auxpass involved VERB AUX [that]
not neg involved VERB PART []
involved ROOT involved VERB VERB [game, is, not, enough, with, .]
enough advmod involved VERB ADV []
with prep involved VERB ADP [playerbase]
it poss playerbase NOUN PRON ['s]
's case it PRON PART []
playerbase pobj with ADP NOUN [it, ,, communication]
, punct playerbase NOUN PUNCT []
zero nummod communication NOUN NUM []
communication conj playerbase NOUN NOUN [zero, ,, approach]
, punct communication NOUN PUNCT []
dubious amod approach NOUN ADJ []
approach appos communication NOUN NOUN [dubious, to]
to prep approach NOUN ADP [fixing]
bug compound fixing NOUN NOUN []
fixing pobj to ADP NOUN [bug, and, balance]
and cc fixing NOUN CCONJ []
balance conj fixing NOUN NOUN []
. punct involv

and cc montage NOUN CCONJ []
tedious conj montage NOUN ADJ [play]
to aux play VERB PART []
play xcomp tedious ADJ VERB [to]
. punct come VERB PUNCT []
This det game NOUN DET []
game nsubj had VERB NOUN [This]
has aux had VERB AUX []
had ROOT had VERB VERB [game, has, tons, doubt, .]
tons dobj had VERB NOUN [of]
of prep tons NOUN ADP [time]
time pobj of ADP NOUN [and, alterations]
and cc time NOUN CCONJ []
alterations conj time NOUN NOUN []
so mark doubt VERB SCONJ []
I nsubj doubt VERB PRON []
doubt advcl had VERB VERB [so, I, is]
the det balance NOUN DET []
balance nsubj is AUX NOUN [the]
is ccomp doubt VERB AUX [balance, bad]
bad acomp is AUX ADJ []
. punct had VERB PUNCT []
I nsubj felt VERB PRON []
felt ccomp Expand VERB VERB [I, giving, …]
like mark giving VERB SCONJ []
I nsubj giving VERB PRON []
really advmod giving VERB ADV []
was aux giving VERB AUX []
n't neg giving VERB PART []
giving advcl felt VERB VERB [like, I, really, was, n't, it, credit, enjoy]
it dative giving VERB P

In [43]:
for sentence in sample_sentences:
    if '\n' in sentence:
        sentence.replace('\n',' ')
    doc = nlp(sentence)
    term = []
    for token in doc:
        if token.pos_ == 'ADJ':
            term.append(token)
            
    print('text :',sentence)
    print('adj :',list(set(term)))
    print('\n')

text : Great game ruined by terrible company that is not involved enough with it's playerbase, zero communication, dubious approach to bug fixing and balance.
adj : [dubious, terrible, Great]


text : After almost 1 year or more since it got out of the beta, the game has released  about 3 major content updates.

The innovation is lackluster if you want a moba that gets constant updates and improvement to their graphics/gameplay/performance / new content, this game is not for you, is the same game it was 2 years ago, literally nothing new maybe 3-4 heroes.

If you like constantAfter almost 1 year or more since it got out of the beta, the game has released  about 3 major content updates.

The innovation is lackluster if you want a moba that gets constant updates and improvement to their graphics/gameplay/performance / new content, this game is not for you, is the same game it was 2 years ago, literally nothing new maybe 3-4 heroes.

If you like constant updates youll better go search ano

In [46]:
aspects = []
for sentence in sentences:
  doc = nlp(sentence)
  descriptive_term = ''
  target = ''
  for token in doc:
    if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
      target = token.text
    if token.pos_ == 'ADJ':
      prepend = ''
      for child in token.children:
        if child.pos_ != 'ADV':
          continue
        prepend += child.text + ' '
      descriptive_term = prepend + token.text
  aspects.append({'aspect': target,
    'description': descriptive_term})
print(aspects)

[{'aspect': '', 'description': 'dubious'}, {'aspect': 'game', 'description': 'constant'}, {'aspect': 'balance', 'description': 'enough'}, {'aspect': 'community', 'description': 'friendly'}, {'aspect': 'curve', 'description': 'free'}, {'aspect': 'game', 'description': 'free'}, {'aspect': 'thing', 'description': 'very negative'}, {'aspect': 'game', 'description': 'superior'}, {'aspect': 'Cons', 'description': 'too much'}, {'aspect': 'base', 'description': 'silent'}, {'aspect': 'game', 'description': 'bored'}, {'aspect': 'community', 'description': 'friendly'}, {'aspect': 'region', 'description': 'entire'}, {'aspect': '', 'description': ''}, {'aspect': 'game', 'description': 'single'}, {'aspect': 'bugs', 'description': 'only'}, {'aspect': 'trees', 'description': 'misleading'}, {'aspect': 'style', 'description': 'incredible'}, {'aspect': '', 'description': 'OFFICIAL'}, {'aspect': 'game', 'description': 'good'}, {'aspect': 'gameplay', 'description': 'as solid'}, {'aspect': 'flaws', 'descrip